# Chains

In [1]:
import os
import warnings
from dotenv import load_dotenv
from pydantic import BaseModel, BaseConfig
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(".env")
api_key = os.environ.get("key")

In [3]:
chat = ChatOpenAI(api_key=api_key, model='gpt-3.5-turbo', max_tokens=500)

In [4]:
human_template = "{question}"
humna_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_template])

In [5]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [6]:
chain.run("What is bit quantization")

'Bit quantization is a process in which a continuous signal is converted into a digital representation by reducing the number of bits used to represent each sample of the signal. It involves dividing the amplitude range of the signal into discrete levels and assigning a specific digital code to each level. The number of bits used for quantization determines the resolution and accuracy of the digital representation. A higher number of bits allows for a finer level of detail and a more accurate representation of the original signal, while a lower number of bits results in a coarser representation with less detail and accuracy. Bit quantization is commonly used in digital audio and video processing, as well as in various other applications involving digitization of analog signals.'

# Simple Sequential Chains

```bash
          --------------------------------------
Input -> |  LLMChain  |  LLMchain  |  LLMchain  | -> Output
          --------------------------------------
```

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template1)
chain_one = LLMChain(llm=chat, prompt=first_prompt)

In [9]:
template2 = "Write a blog post tusing this outline {outline}"
second_prompt = ChatPromptTemplate.from_template(template2)
chain_two = LLMChain(llm = chat, prompt=second_prompt)

In [10]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [11]:
full_chain.run('Skew Data')



> Entering new SimpleSequentialChain chain...
- Introduction to skew data
- Definition of skewness and its importance in data analysis
- Explanation of positive and negative skew
- Examples of skewed data in real-world datasets
- How to identify skewness in data using statistical methods
- Impact of skewness on data analysis and interpretation of results
- Techniques to deal with skewness, such as data transformations and outlier removal
- Discussion on the limitations and considerations when handling skew data
- Conclusion emphasizing the importance of understanding and addressing skewness in data analysis.
Introduction to Skew Data

Data analysis is an integral part of various fields, from finance and marketing to healthcare and scientific research. It involves examining, interpreting, and drawing meaningful insights from datasets. However, not all datasets are created equal, and some exhibit a phenomenon known as skewness. In this blog post, we will explore the concept of skewness

"Introduction to Skew Data\n\nData analysis is an integral part of various fields, from finance and marketing to healthcare and scientific research. It involves examining, interpreting, and drawing meaningful insights from datasets. However, not all datasets are created equal, and some exhibit a phenomenon known as skewness. In this blog post, we will explore the concept of skewness, its importance in data analysis, methods to identify and deal with skewness, and the impact it can have on the interpretation of results.\n\nDefinition of Skewness and Its Importance in Data Analysis\n\nSkewness is a statistical measure that describes the asymmetry or lack of symmetry in a dataset's distribution. It provides insights into the shape and behavior of the data. Understanding skewness is crucial because it can significantly affect the validity of statistical analyses and the interpretation of results. Skewness helps us assess whether the data is normally distributed or if there are any underlyi

# Sequential Chains

In [12]:
from langchain.chains import SequentialChain

In [13]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=chat,
                     prompt=prompt1,
                     output_key="review_summary")

In [14]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=chat,
                     prompt=prompt2,
                     output_key="weaknesses")

In [15]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=chat,
                     prompt=prompt3,
                     output_key="final_plan")

In [16]:
seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['review'],
                            output_variables=['review_summary','weaknesses','final_plan'],
                            verbose=True)

In [17]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [19]:
results = seq_chain(employee_review)



> Entering new SequentialChain chain...

> Finished chain.


In [23]:
print(results['review'])


Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexi

In [24]:
print(results['review_summary'])

Joe Schmo, a Software Engineer, has received a positive performance review. He is praised for his technical expertise, problem-solving skills, collaborative nature, initiative, adaptability, and dedication to self-improvement. However, there are areas for improvement, including time management, written communication skills, and delegation of tasks. Overall, Joe is a highly skilled and valuable team player who would benefit from enhancing these specific areas.


In [25]:
print(results['weaknesses'])

Key employee weaknesses identified in this review summary are time management, written communication skills, and delegation of tasks.


In [26]:
print(results['final_plan'])

Personalized Plan to Address and Fix Key Employee Weaknesses:

1. Time Management:
- Provide time management training: Offer workshops or seminars to improve employee's time management skills. This can include techniques such as prioritizing tasks, setting realistic deadlines, and creating a daily schedule.
- Set clear expectations: Clearly communicate deadlines and expectations for each task or project. This will help the employee understand the importance of effective time management and the consequences of not meeting deadlines.
- Encourage the use of productivity tools: Introduce and encourage the use of time management tools, such as project management software or task management apps. These tools can help the employee stay organized, track progress, and manage their time more efficiently.

2. Written Communication Skills:
- Offer writing workshops or courses: Provide opportunities for the employee to enhance their written communication skills through workshops or online courses. 